In [1]:
%matplotlib inline

import cosima_cookbook as cc
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap, shiftgrid
import numpy as np
import netCDF4 as nc
import xarray as xr
import cmocean as cm
import cartopy
import cartopy.crs as ccrs
import pandas as pd
import copy
import math
from collections import OrderedDict
# import seaborn as sns

from dask.distributed import Client

import sys, os
sys.path.append(os.path.join(os.getcwd(), '..'))  # so we can import ../exptdata
import exptdata
print('Available exptdata keys: ', [k for k in exptdata.exptdict.keys()])

netcdf_index loaded.
Available exptdata keys:  ['1deg', '025deg', '01deg']


In [2]:
client=Client('tcp://10.0.64.12:8786', local_dir='/local/g40/amh157')
client

Client Scheduler: tcp://10.0.64.12:8786 Dashboard: http://10.0.64.12:8787/status,Cluster Workers: 6 Cores: 6 Memory: 24.00 GB


In [3]:
figdir = ''
def savefigure(fname):
    plt.savefig(os.path.join(figdir, fname+'.png'),dpi=300, bbox_inches="tight")  # comment out to disable saving
    plt.savefig(os.path.join(figdir, fname+'.pdf'),dpi=300, bbox_inches="tight")  # comment out to disable saving
    return

In [4]:
def global_scalar(variable):
    for ekey in exptdata.exptdict.keys():
        expt = exptdata.exptdict[ekey]['expt']
        time_units = exptdata.exptdict[ekey]['time_units']
        offset = exptdata.exptdict[ekey]['offset']
        darray = cc.get_nc_variable(expt,'ocean_scalar.nc',
                                    variable,
                                    time_units=time_units,offset=offset)
        annual_average = darray.resample(time='A').mean()
        annual_average.plot(label=exptdata.exptdict[ekey]['desc'])

    plt.gca().autoscale()

In [5]:
def add_cycles():
    cycle_dates = [1718, 1778, 1838, 1898, 1958, 2018]
    ylims=plt.gca().get_ylim()
    for cd in cycle_dates:
        plt.plot([pd.datetime(cd,1,1),pd.datetime(cd,1,1)],ylims,'k--',linewidth=0.8)
    plt.gca().set_ylim(ylims)

In [ ]:
%%time
plt.figure(figsize=(12,8))
plt.subplot(221)
global_scalar('temp_global_ave')
add_cycles()

plt.legend()
plt.xlabel('')
plt.gca().axes.xaxis.set_ticklabels([])
plt.ylabel('Temp (°C)')
plt.title('(a) Global average temperature')


plt.subplot(222)
global_scalar('temp_surface_ave')
add_cycles()

plt.xlabel('')
plt.gca().axes.xaxis.set_ticklabels([])
plt.ylabel('Temp (°C)')
plt.title('(b) Global average surface temperature')

plt.subplot(223)
global_scalar('salt_surface_ave')
add_cycles()

plt.xlabel('')
plt.gca().axes.xaxis.set_ticklabels([])
plt.ylabel('Salinity (g/kg)')
plt.title('(c) Global average surface salinity')

#plt.subplot(223)
#global_scalar('eta_global')
#add_cycles()

#plt.xlabel('Year')
#plt.ylabel('SSH (m)')
#plt.title('(c) Global average sea surface height')

plt.subplot(224)
global_scalar('ke_tot')
add_cycles()

plt.xlabel('Year')
plt.ylabel('KE ($m^2/s^2$)')
plt.title('(d) Global average kinetic energy')

savefigure('GlobalTimeseries')

### Other posible scalar timeseries metrics include:

In [ ]:
global_scalar('temp_surface_ave')
plt.legend()

In [ ]:
global_scalar('pe_tot')
plt.legend()